# Quantitative Macroeconomics - Final Project
Ivana Kaloyanova Ganeva

*December, 2020*

Within these codes, I will seek to implement (at least partially) my idea posed for the Final Project of this course. Due to the time constraints, I will be mostly commenting on the procedure within the *pdf* file of my submission, and comments here will be brief. Therefore, for further reference on the method/variable/parameter choice, please, kindly refer to my report.

I create separate functions for each part of the model and nest them in a single one towards the end to compare different initial conditions/parameterizations in the economy.

In [55]:
# Some technical preliminaries:

# (loading the necessary libraries)
# pip install quantecon # -> run this first if package quantecon is missing
import pandas as pd
import numpy as np
import scipy as sp
from numba import jit
from numpy import vectorize
from itertools import product
from numpy.random import multinomial
from scipy.optimize import fsolve
from scipy.optimize import minimize
from scipy.interpolate import BSpline 
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import seaborn as sns
import quantecon as qe
from quantecon.markov import DiscreteDP
import warnings

warnings.filterwarnings("ignore", category = RuntimeWarning)
np.seterr(divide = 'ignore', invalid = 'ignore')

np.random.seed(5) # for reproducibility of results

In [ ]:
class MyModelHousehold:
    """
    Class that takes the parameters defining the economy and then
    solves the household problem given this parametrization,
    finding the optimal consumption/saving/donation levels, 
    for the observed realizations of the healthcare variables.
    
    Building on/using some codes from Sargent's website:
    https://github.com/QuantEcon/lecture-python.notebooks
    
    An attempt for a general equilibrium
    -> partial is tricky since I assume that the health condition of households determines the TFP param. A
    -> In progress!
    
    Notes
    -----
        Please, kindly refer to the pdf of my submission for further information on 
        the exact choice of parameterization, functional forms, etc.
        I will also include an additional excel file within the folder of my submission.
        
        As in Sargent's codes:
        Need to enumerate the state space S as a sequence S = {0,1,...,n}, where index pairs (a_i, h_i)
        are mapped to s_i according to the rule of
            s_i = a_i * P_size + h_i ,
        and the map is inverted by:
            a_i = s_i // P_size  (integer division)
            h_i = s_i % P_size

    """
    
    def __init__(self,
                 N = 100,                              # number of agents considered in the economy
                 h_num = 4,                            # number of possible health states -> discretizing
                                                       # --- adopting the case of 4 here as in the SILC data:
                                                       # ------ very good, good, fair, bad
                 p_h_0 = (0.168, 0.503, 0.236, 0.077), # respective probabilities of each initial health status
                                                       # --- using the SILC data for Bulgaria, 2019 here + LLN
                                                       # ------ not adding up to 1 exactly, but that is ok for the 
                                                       # ------ numpy.random.multinomial function used
                 p_h_st = (0.22, 0.472, 0.224, 0.068), # respective probabilities of each target health status
                                                       # --- using the SILC data for EU28, 2019 here + LLN
                                                       # ------ not adding up to 1 exactly, but that is ok for the 
                                                       # ------ numpy.random.multinomial function used
                 T = 45,                               # number of periods considered for the simulation
                 P = [[0.8,   0.1,   0.07, 0.03],      # the time-invariant transition matrix for the
                      [0.03,  0.8,   0.1,  0.07],      # Markov Chain component of health
                      [0.03,  0.07,  0.8,  0.1],
                      [0.03,  0.07,  0.1,  0.8]],
                 D_bar = 0,                            # lower bound on the donation fund
                                                       # --- here, a value of 0 means that the fund can be completely
                                                       # --- depleted at any time
                 altruism_coeff = 0.45,                # altruistic coefficient
                                                       # --- set as the ratio of average BG donations to EU donations
                                                       # ------ based on Charities Aid Foundation
                 happy_h_coeff = 0.338,                # happiness from health coefficient
                                                       # --- set as the result of a primitive regression on micro data
                                                       # ------ happiness ~ health
                                                       # ------ based on WVS data from Wave 2005-2009 on Bulgaria
                                                       # ------ using the regression coeff. which tells us
                                                       # ------ (after re-scaling) how much happiness increases after
                                                       # ------ an upwards shift in health (to the upper health state)
                                                       # --- how it enters u could be improved in the future
                 delta = 0.02,                         # depreciation rate
                 mu = 0.3,                             # parameters for the CRRA part of the utility function 
                 gamma = 3,           
                 r = 0.01,                             # interest rate
                 w = 1.0,                              # wages
                 β = 0.96,                             # discount factor
                 a_min = 1e-10,                        # no borrowing limit
                 a_max = 18,                           # max asset level considered for the grid
                 a_size = 100):                        # grid size
        
        # Storing parameter values:
        # -------------------------
        self.N, self.h_num, self.p_h_0, self.p_h_st, self.T = N, h_num, p_h_0, p_h_st, T
        self.r, self.w, self.β = r, w, β
        self.mu, self.gamma = mu, gamma
        self.a_min, self.a_max, self.a_size = a_min, a_max, a_size
        self.D_bar = D_bar
        self.altruism_coeff = altruism_coeff
        self.happy_h_coeff = happy_h_coeff

        # Defining the regions for the h_num # of health states considered:
        # -----------------------------------------------------------------
        self.h_thresh = np.linspace(0, 1, num = h_num + 1)
            # in my case, this is the [0., 0.25, 0.5, 0.75, 1.] vector
        self.h_thresh[0] = self.h_thresh[0] + 0.0001
            # removing the zero case as it is considered death within my model
        self.h_midpoints = self.h_thresh[0:h_num] + slef.h_thresh[1]/2
            # finding the midpoints of the intervals    

        # Generating a grid over a and h:
        # -------------------------------
        self.P = np.asarray(P)
        self.P_vals = np.asarray(self.h_midpoints)
        self.P_size = len(self.h_midpoints)
        
        self.a_vals = np.linspace(a_min, a_max, a_size)
        self.n = a_size * self.P_size

        # Building the arrays of Q and R:
        # -------------------------------
        self.Q = np.zeros((self.n, a_size, self.n))
        self.build_Q()

        self.R = np.empty((self.n, a_size))
        self.build_R()
        
        # Drawing the initial distribution of health:
        # -------------------------------------------
        # Notice that an initial value of h_0_very_good corresponds to the value of self.h_midpoints[3]
        #                                 h_0_good      corresponds to the value of self.h_midpoints[2]
        #                                 h_0_fair      corresponds to the value of self.h_midpoints[1]
        #                                 h_0_poor      corresponds to the value of self.h_midpoints[0]
        # in the current case.
        self.h_midpoints.reverse()
            # thus, reversing the order
        self.h_0_num = multinomial(N, p_h_0, size = 1)
            # this gives a vector with the number of people drawn in each health status group
        self.h_0 = np.repeat(self.h.midpoints, self.h_0_num[0])
            # lastly, this gives the sequence of initial health status for the agents within the economy

    def set_prices(self, r, w):
        """
        Using the method to reset prices: calling it triggers a re-build of R.
        """
        self.r, self.w = r, w
        self.build_R()

    def build_Q(self):
        populate_Q(self.Q, self.a_size, self.P_size, self.P)

    def build_R(self):
        self.R.fill(-np.inf)
        populate_R(self.R, self.a_size, self.P_size, self.a_vals, self.P_vals, self.r, self.w,\
                   self.delta, self.altruism_coeff, self.happy_h_coeff, self.mu, self.gamma, self.h_midpoints)
    
@jit(nopython=True)
def populate_R(R, a_size, P_size, a_vals, P_vals, r, w, delta, altruism_coeff, happy_h_coeff, 
               mu, gamma, h_midpoints):
    n = a_size * P_size
    d_size = a_size; d_vals = a_vals    # the values and sizes of donations
    for d_i in range(d_size):           # running the loop for each donation value possible
        for s_i in range(n):            # and then over all possible values for a and health (starting from highest h)
            a_i = s_i // P_size         # the floor division operator - will change every P_size steps as required
            h_i = s_i % P_size          # the modulo operator - gives a value of the remainder of s_i/P_size
            a = a_vals[a_i]                 
            h = P_vals[h_i]                 # the second element of the P_vals vector
            for new_a_i in range(a_size):
                a_new = a_vals[new_a_i]
                c = w + (1 + r - delta) * a - a_new - d_vals[d_i] # satisfying the B.C. - depleting the resources
                if c > 0:
                    R[d_i, s_i, new_a_i] =  (((c**mu)*((1 - happy_h_coeff*h_midpoints[h_i])**(1 - mu)))**(1 - gamma))/(1 - gamma) +\
                                            altruism_coeff*d_vals[d_i] +\
                                            happy_h_coeff*h_midpoints[h_i]
                                        # The object of utilities for all possible combinations:
                                        # --- first term: usual CRRA
                                        # --- second term: altruistic component
                                        # --- third term: happiness from health -> could be improved
                                        # Also assuming that v(f_t) = v(1-h_t) = nu(h_t) for simplicity
                                        
# Left to do: set-up the constraints of the donation fund;
#             generate health paths - i.e. modify the following function so as it includes the health transfers

@jit(nopython=True)
def populate_Q(Q, a_size, P_size, P):
    n = a_size * P_size
    for s_i in range(n):
        h_i = s_i % z_size
        for a_i in range(a_size):
            for next_z_i in range(z_size):
                Q[s_i, a_i, a_i * z_size + next_z_i] = Π[z_i, next_z_i]


@jit(nopython=True)
def asset_marginal(s_probs, a_size, z_size):
    a_probs = np.zeros(a_size)
    for a_i in range(a_size):
        for z_i in range(z_size):
            a_probs[a_i] += s_probs[a_i * z_size + z_i]
    return a_probs

@jit(nopython=True)
def consumption_marginal(s_probs, a_size, z_size):
    c_probs = np.zeros(a_size)
    for a_i in range(a_size):
        for z_i in range(z_size):
            c_probs[a_i] += s_probs[a_i * z_size + z_i]
    return c_probs


#Parameters
A = 1.0 
N = 1.0  #Normalized labor (to 1)
α = 0.36 #Capital share
β = 0.96
δ = 0.08  #Depreciation


def r_to_w(r):
    """
    Equilibrium wages associated with a given interest rate r.
    """
    return A * (1 - α) * (A * α / (r + δ))**(α / (1 - α))

def rd(K):
    """
    Inverse demand curve for capital.  The interest rate associated with a
    given demand for capital K.
    """
    return A * α * (N / K)**(1 - α) - δ


def prices_to_capital_stock(am, r):
    """
    Map prices to the induced level of capital stock.
    
    Parameters:
    ----------
    
    am : Household
        An instance of an aiyagari_household.Household 
    r : float
        The interest rate
    """
    w = r_to_w(r)
    am.set_prices(r, w)
    aiyagari_ddp = DiscreteDP(am.R, am.Q, β)
    # Compute the optimal policy
    results = aiyagari_ddp.solve(method='policy_iteration')
    # Compute the stationary distribution
    stationary_probs = results.mc.stationary_distributions[0]
    # Extract the marginal distribution for assets
    asset_probs = asset_marginal(stationary_probs, am.a_size, am.z_size)
    # Return K
    return np.sum(asset_probs * am.a_vals)

# Create an instance of Household
am = Household(a_max = 30)

# Use the instance to build a discrete dynamic program
am_ddp = DiscreteDP(am.R, am.Q, am.β)

# Create a grid of r values at which to compute demand and supply of capital
num_points = 20
r_vals = np.linspace(0.005, 0.04, num_points)

# Compute supply of capital
k_vals = np.empty(num_points)
for i, r in enumerate(r_vals):
    k_vals[i] = prices_to_capital_stock(am, r)

# Plot against demand for capital by firms
fig, ax = plt.subplots()
ax.plot(k_vals, r_vals, lw=2, alpha=0.6, label='supply of capital')
ax.plot(k_vals, rd(k_vals), lw=2, alpha=0.6, label='demand for capital')
ax.set_xlabel('capital')
ax.set_ylabel('rate of return')
ax.legend(loc='upper right')
plt.title('Graph 3: Supply and demand for capital')

plt.show()


In [58]:
aaaa = (1,2,3,4)
op = (0.168, 0.503, 0.236, 0.077)
bbbb = multinomial(100, op , size = 1)
1%4

1